In [1]:
import sys
!"{sys.executable}" -m pip install pandas matplotlib numpy scikit-learn xgboost



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\stanl\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier

# ======================================================
# LOAD DATA
# ======================================================
train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col])

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# ======================================================
# FIXED BEST PARAMETERS (from your first successful tuning)
# ======================================================
best_params = {
    "eta": 0.05,
    "max_depth": 4,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "n_estimators": 600
}

# ======================================================
# 5-FOLD CROSS-VALIDATION FOR LOG-LOSS
# ======================================================
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
losses = []

for train_idx, val_idx in kf.split(X_scaled, y):

    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = XGBClassifier(
        eta=best_params["eta"],
        max_depth=best_params["max_depth"],
        subsample=best_params["subsample"],
        colsample_bytree=best_params["colsample_bytree"],
        n_estimators=best_params["n_estimators"],
        eval_metric="logloss",
        use_label_encoder=False
    )

    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:,1]
    losses.append(log_loss(y_val, preds))

avg_loss = np.mean(losses)
print(f"\nAverage Log-Loss with fixed best hyperparameters = {avg_loss:.5f}")

# ======================================================
# FINAL MODEL TRAINING
# ======================================================
final_model = XGBClassifier(
    eta=best_params["eta"],
    max_depth=best_params["max_depth"],
    subsample=best_params["subsample"],
    colsample_bytree=best_params["colsample_bytree"],
    n_estimators=best_params["n_estimators"],
    eval_metric="logloss",
    use_label_encoder=False
)

final_model.fit(X_scaled, y)

# ======================================================
# PREDICT TEST & EXPORT CSV
# ======================================================
test_preds = final_model.predict_proba(test_scaled)[:,1]

output = pd.DataFrame({
    "y_passXtremeDurability": test_preds
})

output.to_csv("XGBOOST_predictions.csv", index=False)
print("\nCSV file 'XGBOOST_predictions.csv' created successfully!")


C:\Users\stanl\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:45:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\stanl\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:45:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\stanl\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:45:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\stanl\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:45:49] WARNING: C:\actio


Average Log-Loss with fixed best hyperparameters = 0.42542


C:\Users\stanl\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:45:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



CSV file 'XGBOOST_predictions.csv' created successfully!
